In [1]:
import pandas as pd, numpy as np
import lbwsg
import lbwsg_boe
import lsff_interventions

!whoami
!date

ndbs
Wed Dec  2 15:03:04 PST 2020


In [2]:
%load_ext autoreload
%autoreload 2

# Define artifact path

In [3]:
artifact_dir = '/share/costeffectiveness/artifacts/vivarium_conic_lsff/'
artifact_path = artifact_dir+'nigeria.hdf'

# Create a Calculator object, initialize baseline and intervention populations of 100 simulants, and compute the PIF

Ths PIF is very small, and only shows up in one of three draws with a population of 100.

In [68]:
# Use 3 draws for testing
draws = [0,50,100]

In [27]:
calc = lbwsg_boe.IronBirthweightCalculator('Nigeria', artifact_path, 2017, draws)

In [28]:
bp, ip = calc.initialize_population_tables(100)

In [29]:
bp.head()

sex  age  age_start   age_end  \
draw simulant_id                                     
0    0              Male  0.0        0.0  0.019178   
     1              Male  0.0        0.0  0.019178   
     2              Male  0.0        0.0  0.019178   
     3            Female  0.0        0.0  0.019178   
     4            Female  0.0        0.0  0.019178   

                  iron_fortification_propensity lbwsg_cat  ga_propensity  \
draw simulant_id                                                           
0    0                                 0.720809     cat44       0.242089   
     1                                 0.945052     cat44       0.614174   
     2                                 0.430046     cat55       0.909457   
     3                                 0.801278     cat56       0.452752   
     4                                 0.909069     cat55       0.044344   

                  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                
0    0                 0.498989        38.484178  2749.494647   
     1                 0.078850        39.228348  2539.424849   
     2                 0.173870        41.818914  3586.935053   
     3                 0.886064        40.905504  4443.031969   
     4                 0.481049        40.088688  3740.524390   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
0    0                              2749.072562                     False   
     1                              2539.002764                     False   
     2                              3586.512968                     False   
     3                              4442.609885                     False   
     4                              3740.102305                     False   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
0    0                           0.0                0.0          2749.072562   
     1                           0.0                0.0          2539.002764   
     2                           0.0                0.0          3586.512968   
     3                           0.0                0.0          4442.609885   
     4                           0.0                0.0          3740.102305   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
0    0                       cat44             2.880156  
     1                       cat44             2.880156  
     2                       cat55             1.000000  
     3                       cat56             1.000000  
     4                       cat55             1.000000

In [30]:
pif = lbwsg_boe.population_impact_fraction(bp, ip, 'lbwsg_relative_risk')
pif

draw
0      0.000021
50     0.000000
100    0.000000
Name: lbwsg_relative_risk, dtype: float64

## Hmm, sometimes the birthweight is larger in baseline than intervention

This must be because we're not using common random numbers.

In [31]:
ip.loc[ip.treated_birthweight>bp.treated_birthweight].shape

(77, 17)

In [32]:
ip.loc[ip.treated_birthweight<bp.treated_birthweight].shape

(223, 17)

In [33]:
ip.treated_birthweight.mean()

3208.273029199195

In [34]:
bp.treated_birthweight.mean()

3205.881646408204

In [35]:
bp.mother_is_iron_fortified.sum()

35

In [36]:
ip.mother_is_iron_fortified.sum()

87

# Check the runtime for some larger populations

The code is surprisingly slow for large populations...

I'm guessing the slowest piece will be the `apply_birthweight_shift` function in the `LBWSGDistribution` class, but I haven't tested this.

In [42]:
%timeit calc.initialize_population_tables(100)

221 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%timeit calc.initialize_population_tables(1000)

391 ms ± 15.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%timeit calc.initialize_population_tables(10_000)

2.03 s ± 17.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%timeit calc.initialize_population_tables(100_000)

21.5 s ± 340 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Estimate runtime per draw per 100,000 simulants

Based on the largest population, it looks like it's about 7 seconds per draw per 100,000 simulants

In [61]:
21.5/3

7.166666666666667

In [62]:
2.03/3*10

6.766666666666667

In [64]:
391/1000/3*100

13.033333333333333

In [65]:
# How many minutes would 1000 draws take with 100,000 simulants?
7000/60

116.66666666666667

# See if the PIF is more detectable with a larger population of 10,000

Hmm, now it's **negative** in one of the draws.

In [40]:
bp1, ip1 = calc.initialize_population_tables(10_000)

In [46]:
pif = lbwsg_boe.population_impact_fraction(bp1,ip1,'lbwsg_relative_risk')
pif

draw
0     -0.000096
50     0.000908
100    0.000594
Name: lbwsg_relative_risk, dtype: float64

In [47]:
pif.mean()

0.0004689621877009763

In [49]:
idx = pd.IndexSlice

## Check the mean birthweights in draw 0

It's larger in the intervention scenario as expected. By about 3g, which is a bit smaller than expected for Nigeria, but within a reasonable range.

In [50]:
bp1.loc[idx[0,:],'treated_birthweight'].mean()

3174.695806041431

In [51]:
ip1.loc[idx[0,:],'treated_birthweight'].mean()

3177.036278353212

In [55]:
bp1.loc[idx[0,:],:].head()

sex  age  age_start   age_end  \
draw simulant_id                                     
0    0              Male  0.0        0.0  0.019178   
     1              Male  0.0        0.0  0.019178   
     2              Male  0.0        0.0  0.019178   
     3            Female  0.0        0.0  0.019178   
     4              Male  0.0        0.0  0.019178   

                  iron_fortification_propensity lbwsg_cat  ga_propensity  \
draw simulant_id                                                           
0    0                                 0.063869     cat56       0.970781   
     1                                 0.916032     cat52       0.811783   
     2                                 0.263193     cat55       0.575768   
     3                                 0.400943     cat50       0.227377   
     4                                 0.674938     cat52       0.337057   

                  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                
0    0                 0.489573        41.941562  4244.786639   
     1                 0.703595        39.623565  3351.797556   
     2                 0.880639        41.151535  3940.319261   
     3                 0.649608        37.227377  3824.803964   
     4                 0.568341        38.674115  3284.170562   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
0    0                              4244.364142                      True   
     1                              3351.375059                     False   
     2                              3939.896764                     False   
     3                              3824.381467                     False   
     4                              3283.748064                     False   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
0    0                     75.991462            2.86891          4247.233053   
     1                      0.000000            0.00000          3351.375059   
     2                      0.000000            0.00000          3939.896764   
     3                      0.000000            0.00000          3824.381467   
     4                      0.000000            0.00000          3283.748064   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
0    0                       cat56             1.000000  
     1                       cat52             1.461914  
     2                       cat55             1.000000  
     3                       cat50             2.060782  
     4                       cat52             1.461914

In [52]:
ip1.loc[idx[0,:],:].head()

sex  age  age_start   age_end  \
draw simulant_id                                     
0    0              Male  0.0        0.0  0.019178   
     1              Male  0.0        0.0  0.019178   
     2              Male  0.0        0.0  0.019178   
     3            Female  0.0        0.0  0.019178   
     4              Male  0.0        0.0  0.019178   

                  iron_fortification_propensity lbwsg_cat  ga_propensity  \
draw simulant_id                                                           
0    0                                 0.063869     cat56       0.970781   
     1                                 0.916032     cat52       0.811783   
     2                                 0.263193     cat55       0.575768   
     3                                 0.400943     cat50       0.227377   
     4                                 0.674938     cat52       0.337057   

                  bw_propensity  gestational_age  birthweight  \
draw simulant_id                                                
0    0                 0.489573        41.941562  4244.786639   
     1                 0.703595        39.623565  3351.797556   
     2                 0.880639        41.151535  3940.319261   
     3                 0.649608        37.227377  3824.803964   
     4                 0.568341        38.674115  3284.170562   

                  treatment_deleted_birthweight  mother_is_iron_fortified  \
draw simulant_id                                                            
0    0                              4243.681346                      True   
     1                              3350.692263                     False   
     2                              3939.213968                     False   
     3                              3823.698671                     False   
     4                              3283.065268                     False   

                  mother_daily_flour  birthweight_shift  treated_birthweight  \
draw simulant_id                                                               
0    0                    217.358697          21.274192          4264.955539   
     1                      0.000000           0.000000          3350.692263   
     2                      0.000000           0.000000          3939.213968   
     3                      0.000000           0.000000          3823.698671   
     4                      0.000000           0.000000          3283.065268   

                 treated_lbwsg_cat  lbwsg_relative_risk  
draw simulant_id                                         
0    0                       cat56             1.000000  
     1                       cat52             1.461914  
     2                       cat55             1.000000  
     3                       cat50             2.060782  
     4                       cat52             1.461914

## As indicated by PIF, mean RR in baseline is slightly lower - rounding error or real difference?

It looks like it's a real difference, based on the category counts below. Why would RR's in baseline be lower than in intervention?

The only explanation is that it is random variability in the mother flour consumption value, dose-response value, and iron concentration value, between baseline scenario and intervention scenario.

I think the only way to get rid of this variability is to use common random numbers to re-use values that should be the same between baseline and intervention.

In [53]:
bp1.loc[idx[0,:],'lbwsg_relative_risk'].mean()

11.598937025989354

In [54]:
ip1.loc[idx[0,:],'lbwsg_relative_risk'].mean()

11.60004986805275

In [74]:
bp1_counts = bp1.loc[idx[0,:], 'treated_lbwsg_cat'].value_counts()
ip1_counts = ip1.loc[idx[0,:], 'treated_lbwsg_cat'].value_counts()

In [75]:
bp1_counts

cat52     1809
cat44     1373
cat54     1112
cat51      932
cat55      747
cat42      556
cat43      455
cat53      437
cat48      417
cat56      338
cat50      162
cat37      146
cat32      140
cat40      110
cat35      103
cat49       87
cat33       68
cat45       67
cat34       66
cat39       59
cat117      57
cat28       54
cat24       49
cat36       41
cat27       37
cat17       37
cat30       36
cat29       35
cat31       34
cat41       31
cat88       29
cat95       29
cat21       27
cat10       24
cat47       24
cat26       22
cat81       22
cat25       21
cat23       20
cat116      19
cat22       19
cat123      17
cat20       14
cat38       13
cat96       12
cat89       12
cat19       10
cat82       10
cat46       10
cat90        9
cat106       9
cat15        9
cat80        8
cat14        8
cat11        8
Name: treated_lbwsg_cat, dtype: int64

In [76]:
ip1_counts

cat52     1818
cat44     1363
cat54     1107
cat51      928
cat55      749
cat42      556
cat43      453
cat53      443
cat48      417
cat56      343
cat50      162
cat37      147
cat32      141
cat40      109
cat35      103
cat49       87
cat45       68
cat33       66
cat34       64
cat39       59
cat117      56
cat28       55
cat24       49
cat36       41
cat27       37
cat17       37
cat29       36
cat30       35
cat31       32
cat95       31
cat41       31
cat21       28
cat88       28
cat10       24
cat47       23
cat81       22
cat26       22
cat25       20
cat23       20
cat116      19
cat22       19
cat123      18
cat20       14
cat38       13
cat89       13
cat96       12
cat46       11
cat19       10
cat82       10
cat106      10
cat90        9
cat80        8
cat14        8
cat11        8
cat15        8
Name: treated_lbwsg_cat, dtype: int64

# Try calculating PIF with an even larger population (100,000)

Well, this time at least it's positive in all draws. Woo hoo!

In [66]:
bp2, ip2 = calc.initialize_population_tables(100_000)

In [67]:
pif = lbwsg_boe.population_impact_fraction(bp2,ip2,'lbwsg_relative_risk')
pif

draw
0      0.000260
50     0.000285
100    0.000607
Name: lbwsg_relative_risk, dtype: float64